In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 读取数据库
df = pd.read_csv('./data/tianchi_ali/tianchi_mobile_recommend_train_user.csv', usecols=[0, 2, 4, 5])
df.columns = ['userId', 'behav', 'itemCat', 'timestp']
df['userId'] = df['userId'].astype('uint32')
df['behav'] = df['behav'].astype('uint8')
df['itemCat'] = df['itemCat'].astype('uint32')
df['timestp'] = df['timestp'].str.replace('-','').str.replace(' ', '').astype(int).astype('uint32') // 100

clone = df.copy()

df.dtypes

userId     uint32
behav       uint8
itemCat    uint32
timestp    uint32
dtype: object

In [3]:
# 计算内存占用，单位KB
df = clone.copy()

df.memory_usage() // 1024

Index          0
userId     47878
behav      11969
itemCat    47878
timestp    47878
dtype: int64

In [4]:
# behav包含四种行为 
# 1--浏览 
# 2--收藏 
# 3--加购物车 
# 4--购买
df

,userId,behav,itemCat,timestp
0,98047837,1,4245,20141206
1,97726136,1,5894,20141209
2,98607707,1,2883,20141218
3,98662432,1,6562,20141206
4,98145908,1,13926,20141216
...,...,...,...,...
12256901,93812622,1,11,20141213
12256902,93812622,1,12311,20141214
12256903,93812622,1,8765,20141211
12256904,93812622,1,7951,20141208


In [5]:
# 提取数据各类别全集，同时也作为后续数据的索引列表
userId = list(df['userId'].unique())
itemCat = list(df['itemCat'].unique())
timestp = sorted(list(df['timestp'].unique()))
print(f'userId:  [{len(userId)}]\nitemCat: [{len(itemCat)}]\ntimestp: [{len(timestp)}]')

userId:  [10000]
itemCat: [8916]
timestp: [31]


In [6]:
# 构建三维数组
shape = (len(userId), len(itemCat), len(timestp))
input = np.zeros(shape, dtype=np.float32)
size = input.nbytes
print(f'input array storage: {size} B = {size // 1024} KB = {size // 1024 // 1024} MB = {size // 1024 // 1024 // 1024} GB')

input array storage: 11055840000 B = 10796718 KB = 10543 MB = 10 GB


In [7]:
# 定义时间衰减函数
alpha = 0.23 # 超参数，衰减因子，控制每一天的衰减程度，0.23大约是3日便会衰减一半
def decay_func(x):
    return np.exp(-x * alpha)

In [8]:
# 提取最后一天的交互
last_day = timestp[-1]

interact = df[df['timestp'] == last_day] # 提取df中最后一天的所有交互
interact = interact.drop(columns=['behav', 'timestp']).drop_duplicates() # 删除behav，timestp列

# 在df中剔除最后一天的交互
df = df[df['timestp'] != last_day]

df

,userId,behav,itemCat,timestp
0,98047837,1,4245,20141206
1,97726136,1,5894,20141209
3,98662432,1,6562,20141206
4,98145908,1,13926,20141216
5,93784494,1,3979,20141203
...,...,...,...,...
12256901,93812622,1,11,20141213
12256902,93812622,1,12311,20141214
12256903,93812622,1,8765,20141211
12256904,93812622,1,7951,20141208


In [9]:
interact

,userId,itemCat
2,98607707,2883
33,103802946,11406
38,101781721,9829
79,104221274,5399
130,101260672,3424
...,...,...
12255140,26920547,6054
12255679,56708793,4370
12255946,56708793,4254
12256403,66961542,1083


In [10]:
# 计算每个用户的总交互次数
N = list() # 记录用户总交互次数
for i in range(len(userId)):
    N.append(0)
len(N)

10000

In [11]:
# 复制一个df,日期，用户，商品均为一致的行保留一个
no_behav = df.copy().drop(columns=['behav']).drop_duplicates().reset_index().drop(columns=['index'])
no_behav

,userId,itemCat,timestp
0,98047837,4245,20141206
1,97726136,5894,20141209
2,98662432,6562,20141206
3,98145908,13926,20141216
4,93784494,3979,20141203
...,...,...,...
1805124,76314785,9291,20141203
1805125,76314785,7582,20141203
1805126,76314785,9720,20141209
1805127,76314785,8095,20141206


In [12]:
# 计算总交互次数
for index, row in no_behav.iterrows():
    if (index % 100000 == 0):
        print(f'processing [{index}/{len(no_behav)}]')
    id = row.userId
    idx = userId.index(id)
    N[idx] += 1

processing [0/1805129]
processing [100000/1805129]
processing [200000/1805129]
processing [300000/1805129]
processing [400000/1805129]
processing [500000/1805129]
processing [600000/1805129]
processing [700000/1805129]
processing [800000/1805129]
processing [900000/1805129]
processing [1000000/1805129]
processing [1100000/1805129]
processing [1200000/1805129]
processing [1300000/1805129]
processing [1400000/1805129]
processing [1500000/1805129]
processing [1600000/1805129]
processing [1700000/1805129]
processing [1800000/1805129]


In [13]:
# 计算用户平均每日交互次数
for i in range(len(userId)):
    N[i] /= len(timestp)-1
    N[i] += 1
    N[i] = int(N[i])

In [14]:
# 填充input数组
# 记录开始前的时间
import time

last_time = int(time.time())
# 填充input数组
for index, row in df.iterrows():
    # 进度输出
    if index % 100000 == 0:
        this_time = int(time.time())
        print(f'loop {index}/{len(df)} || time_spent: [{this_time-last_time}s]')
        last_time = this_time
        
    # input[u][c][t]
    u = userId.index(row.userId)
    c = itemCat.index(row.itemCat)
    t = timestp.index(row.timestp)
    behav = row.behav
    if behav == 3:
        score = 2
    else:
        score = behav
    # print(f'user: {u}, cata: {c}, time: {t}, score: {score}')

    # 将数据填入数组
    input[u, c, t] = input[u, c, t] + score

    # 计算时间衰减
    for day in range(t+1, len(timestp)):
        input[u, c, day] = input[u, c, day] + score * decay_func(day-t)

loop 0/11881309 || time_spent: [0s]
loop 100000/11881309 || time_spent: [10s]
loop 200000/11881309 || time_spent: [12s]
loop 300000/11881309 || time_spent: [13s]
loop 400000/11881309 || time_spent: [13s]
loop 500000/11881309 || time_spent: [15s]
loop 600000/11881309 || time_spent: [16s]
loop 800000/11881309 || time_spent: [35s]
loop 900000/11881309 || time_spent: [19s]
loop 1000000/11881309 || time_spent: [21s]
loop 1100000/11881309 || time_spent: [22s]
loop 1200000/11881309 || time_spent: [22s]
loop 1300000/11881309 || time_spent: [19s]
loop 1400000/11881309 || time_spent: [17s]
loop 1500000/11881309 || time_spent: [17s]
loop 1600000/11881309 || time_spent: [16s]
loop 1700000/11881309 || time_spent: [15s]
loop 1800000/11881309 || time_spent: [17s]
loop 1900000/11881309 || time_spent: [16s]
loop 2000000/11881309 || time_spent: [17s]
loop 2100000/11881309 || time_spent: [17s]
loop 2200000/11881309 || time_spent: [17s]
loop 2300000/11881309 || time_spent: [16s]
loop 2400000/11881309 || t

In [15]:
# 输出评价分数据（测试用）
slice = pd.DataFrame(input[430, :, :]) # 输出user的切片
slice = slice.loc[~(slice == 0).all(axis=1)] # 输出除了全为0的行
slice.T.to_csv('./data/test_data.csv', index=False)

slice

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,15.000000,11.918005,9.469255,7.523642,5.977786,4.749552,3.773678
1,0.0,0.000000,2.000000,1.589067,1.262567,1.003152,0.797038,0.633274,0.503157,0.399775,...,1.601485,1.272434,1.010992,0.803267,0.638222,0.507089,0.402899,0.320117,0.254344,0.202085
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,11.918005,9.469255,...,0.599326,0.476185,0.378345,0.300608,0.238843,0.189769,0.150778,0.119798,0.095183,0.075626
6,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.794534,2.631284,2.090643,1.661086,...,0.105133,0.083532,0.066369,0.052732,0.041898,0.033289,0.026449,0.021015,0.016697,0.013266
11,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,15.722688,12.492207,42.925461,34.105736,27.098148,21.530386,17.106619,13.591784,10.799129,8.580267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2825,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.794534,0.631284,...,0.039955,0.031746,0.025223,0.020041,0.015923,0.012651,0.010052,0.007987,0.006346,0.005042
2842,2.0,1.589067,1.262567,1.003152,0.797038,0.633274,0.503157,0.399775,0.317635,0.252372,...,0.015973,0.012691,0.010084,0.008012,0.006366,0.005058,0.004018,0.003193,0.002537,0.002016
3141,2.0,1.589067,1.262567,1.003152,0.797038,0.633274,0.503157,0.399775,0.317635,0.252372,...,0.015973,0.012691,0.010084,0.008012,0.006366,0.005058,0.004018,0.003193,0.002537,0.002016
3287,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,8.000000,6.356268,5.050270,4.012609,3.188152,2.533094,2.012628,1.599101


In [134]:
import torch
from torch import nn

input.shape

(10000, 8916, 31)

In [135]:
# 定义超参数
class Para():
    input_size = 1
    hidden_size = 64
    output_size = 1
    num_layers = 1
    lr = 0.001
    epochs = 50
    threshold = 0.2
    step = 0.05
    batch = 9900 # 调整一次训练的用户数,经过观察，9900后的用户行为是异常数据，因而忽略

para = Para()

In [136]:
# 定义迭代器
class InputIterator():
    def __init__(self, C):
        self.C = C
        self.u, self.c, self.t = C.shape
        self.cur = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.cur < self.u:
            ret = self.C[self.cur]
            self.cur += 1
            return ret
        else:
            raise StopIteration

iterator = InputIterator(input)

In [137]:
# 定义模型
import torch
from torch import nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__() # 调用父类nn.Module中的init方法
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0=None):
        # 输入x的形状为（batch_size，seq_len，input_size）
        batch_size, seq_len, input_size = x.shape

        # h0默认为全0，否则取h0输入模型的值

        # 输入rnn的形状为（1，seq_len，input_size）
        # 输出rnn的形状为（1，seq_len，hidden_size）
        out, _ = self.rnn(x, h0)
        # 输入fc的形状为（1，1，hidden_size）（只取最后一个时间步）
        # 输出fc的形状为（1，1，output_size）
        out = self.fc(out[:,-1,:])
        return out

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRU, self).__init__() # 调用父类nn.Module中的init方法
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0=None):
        # 输入x的形状为（batch_size，seq_len，input_size）
        batch_size, seq_len, input_size = x.shape

        # h0默认为全0，否则取h0输入模型的值

        # 输入rnn的形状为（1，seq_len，input_size）
        # 输出rnn的形状为（1，seq_len，hidden_size）
        out, _ = self.gru(x, h0)
        # 输入fc的形状为（1，1，hidden_size）（只取最后一个时间步）
        # 输出fc的形状为（1，1，output_size）
        out = self.fc(out[:,-1,:])
        return out

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTM, self).__init__() # 调用父类nn.Module中的init方法
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True) # LSTM考虑计算复杂度只使用1层
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0=None):
        # 输入x的形状为（batch_size，seq_len，input_size）
        batch_size, seq_len, input_size = x.shape

        # h0默认为全0，否则取h0输入模型的值

        # 输入rnn的形状为（1，seq_len，input_size）
        # 输出rnn的形状为（1，seq_len，hidden_size）
        out, _ = self.lstm(x, h0)
        # 输入fc的形状为（1，1，hidden_size）（只取最后一个时间步）
        # 输出fc的形状为（1，1，output_size）
        out = self.fc(out[:,-1,:])
        return out

In [138]:
from sklearn.preprocessing import MinMaxScaler

# 数据处理部分
# 输入一个dataframe
# 返回列数cat，行数seq，商品全集item_u，模型输入input
def preprocessing(data):
    seq, cat = data.shape # 获取data形状
    item_u = data.columns.values.tolist() # 获取商品全集
    scaler = MinMaxScaler()
    data_out = scaler.fit_transform(data.to_numpy()) # 数据归一化
    data_out = data_out.reshape(1, seq, cat) # reshape
    
    return cat, seq, item_u, data_out

In [139]:
# 划分数据集
# 输入数据部分，及分割线
# 输出分割后的训练及测试集
def separate(data, sep_line=20):
    train = data[:, sep_line:, :]
    test = data[:, :sep_line, :]
    
    train_x = train[:, :-1, :]
    train_y = train[:, -1:, :]
    test_x = test[:, :-1, :]
    test_y = test[:, -1, :]

    # 将训练集及测试集转换为tensor
    train_x = torch.tensor(train_x, dtype=torch.float32)
    train_y = torch.tensor(train_y, dtype=torch.float32)
    test_x = torch.tensor(test_x, dtype=torch.float32)
    test_y = torch.tensor(test_y, dtype=torch.float32)
    test_y = test_y.reshape(1, 1, cat)
    
    return train_x, train_y, test_x, test_y

In [140]:
# 训练模型
# 输入训练集和超参数
# 输出训练后的模型
def train(train_x, train_y, para):
    # 定义模型
    model = RNN(para.input_size, para.hidden_size, para.output_size, para.num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), para.lr)

    # 训练模型
    for epoch in range(para.epochs):
        optimizer.zero_grad() # 优化器梯度清零
        output = model(train_x) # model的output会是一个形状为（1，1，output_size）的一维向量
        output = output.reshape(1, 1, para.output_size) # reshape output的形状（不reshape也行，就是会一直报警告而已）
        loss = criterion(output, train_y) # 计算预测输出与实际输出的loss
        loss.backward() # 计算梯度
        optimizer.step() # 更新参数

    return model

In [141]:
# 测试模型
# 输入测试集及与用户相关的商品类别集
# 输出测试结果的分析值
def test(test_x, test_y, last_day, para, item_u, N, cat):
    criterion = nn.MSELoss()
    
    # 模型评估
    with torch.no_grad():
        output = model(test_x)
        output = output.reshape(1, 1, cat)
        test_loss = criterion(output, test_y)
        
    # 调整数据结构
    output = np.array(output).reshape(cat)

    # 取得阈值
    threshold = para.threshold

    # 通过阈值与平均交易数得到可能交互的项目
    while True:
        count = cat
        for i in range(len(output)):
            if output[i] < threshold:
                output[i] = -1
                count -= 1
        
        if count <= N:
            break;
        else:
            threshold += para.step

    # 定义真假正负例记录数组
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for i in range(cat): 
        if output[i] > 0: # 正例
            if int(item_u[i]) in last_day: # 真正例
                TP += 1
            else: # 假正例
                FP += 1
        else: # 负例
            if int(item_u[i]) in last_day: # 假负例
                FN += 1
            else: # 真负例
                TN += 1

    # 计算各类标签
    pos = TP + FP
    neg = TN + FN
    
    accuracy = round((TP + TN) * 100 / (pos + neg), 2) # 准确率
    try:
        precision = round(TP * 100 / pos, 2) # 精准度
    except ZeroDivisionError:
        precision = 0
    try:
        recall = round(TP * 100 / (TP + FN), 2) # 召回率
    except ZeroDivisionError:
        recall = 0
    try:
        specificity = round(TN * 100 / (TN + FP), 2) # 特异度
    except ZeroDivisionError:
        specificity = 0
    if (precision == 0 or recall == 0):
        f1 = 0
    else:
        f1 = round((2 * precision * recall / 100) / (precision + recall), 2) # F1 score
    
    return test_loss, accuracy, precision, recall, specificity, f1

In [142]:
# 记录训练开始时间
start = time.time()

In [143]:
# 循环遍历每个用户的商品-时间矩阵
cur = 0
avg_loss = 0
avg_acc = 0
avg_pre = 0
avg_rec = 0
avg_spec = 0
avg_f1 = 0

for data in iterator:
    # 清理无用数据
    data = pd.DataFrame(data)
    data = data.loc[~(data == 0).all(axis=1)].T # 清除空行，并转置矩阵

    # 数据处理
    try:
        cat, seq, item_u, data_out = preprocessing(data) # 在读取大量数据时，有时data会出错，因此catch错误并跳过出错的数据
    except ValueError:
        print(f'[[{cur+1}/10000]] loop skipped')
        cur += 1
        para.batch += 1
        continue
    para.input_size = cat
    para.output_size = cat

    # 划分数据集
    # 训练用户集前30天的兴趣分，预测用户在第31天的兴趣分，从而计算模型准确率
    # 训练前20天，测试后11天
    train_x, train_y, test_x, test_y = separate(data_out, 20)

    model = train(train_x, train_y, para)

    # 得到用户最后一天实际交互了的商品集
    last_day = list()
    for row in interact.itertuples():
        if row.userId == userId[cur]:
            last_day.append(itemCat.index(row.itemCat))
    
    loss, accuracy, precision, recall, specificity, f1 = test(test_x, test_y, last_day, para, item_u, N[cur], cat)

    print(f'[{cur+1}/{para.batch}] final loss [{loss}] accuracy[{accuracy}] precision[{precision}] recall[{recall}] specificity[{specificity}] F1 score[{f1}]')

    avg_loss += loss
    avg_acc += accuracy
    avg_pre += precision
    avg_rec += recall
    avg_spec += specificity
    avg_f1 += f1
    
    cur += 1 # 用户记录

    if cur == para.batch:
        break


[1/5000] final loss [0.22762928903102875] accuracy[83.7] precision[15.38] recall[15.38] specificity[90.98] F1 score[0.15]
[2/5000] final loss [0.07588911056518555] accuracy[95.77] precision[0.0] recall[0.0] specificity[97.14] F1 score[0]
[3/5000] final loss [0.10331607609987259] accuracy[94.4] precision[12.5] recall[100.0] specificity[94.35] F1 score[0.22]
[4/5000] final loss [0.025702161714434624] accuracy[85.71] precision[0.0] recall[0] specificity[85.71] F1 score[0]
[5/5000] final loss [0.03648802638053894] accuracy[100.0] precision[0] recall[0] specificity[100.0] F1 score[0]
[6/5000] final loss [0.10856612771749496] accuracy[95.19] precision[0.0] recall[0.0] specificity[95.65] F1 score[0]
[7/5000] final loss [0.04397798329591751] accuracy[80.0] precision[0.0] recall[0] specificity[80.0] F1 score[0]
[8/5000] final loss [0.22684812545776367] accuracy[96.3] precision[50.0] recall[75.0] specificity[97.12] F1 score[0.6]
[9/5000] final loss [0.2256808876991272] accuracy[97.06] precision[

In [144]:
# 模型评估
avg_loss /= para.batch
avg_acc /= para.batch
avg_pre /= para.batch
avg_rec /= para.batch
avg_spec /= para.batch
avg_f1 /= para.batch
avg_time = time.time() - start
avg_time /= para.batch

In [145]:
# 输出结果
print(f'loss[{avg_loss}]\naccuracy[{avg_acc}]\nprecision[{avg_pre}]\nrecall[{avg_rec}]\nspecificity[{avg_spec}]\nf1 score[{avg_f1}]\naverage time[{avg_time}]')

loss[0.12462495267391205]
accuracy[91.60157779109181]
precision[8.140631116437012]
recall[9.926315158777735]
specificity[94.3477411623725]
f1 score[0.07620131815458374]
average time[0.1982944823558968]
